# Run on IBMQ real device

**Important**
- `pip install pennylane-qiskit` before launching.
- Create `./config.toml` file, see [PennyLane Configuration File](https://docs.pennylane.ai/en/latest/introduction/configuration.html#format) for further detail.

This script runs the pretrained QCGNN on IBMQ real device with the default `shots` is 1024.

### Import packages

In [ ]:
import os
from itertools import product

import lightning as L
import numpy as np
import torch
from tqdm import tqdm

import g_main
import module_gmail
import module_training

# Output directory.
general_config = g_main.json_config.copy()
ibmq_dir = os.path.join(general_config["predictions_dir"], "ibmq")
os.makedirs(ibmq_dir, exist_ok=True)

# Check `config.toml` file
if not os.path.isfile("config.toml"):
    raise FileNotFoundError(
        "`./config.toml` file (containing IBMQ account token) not found, "
        "see https://docs.pennylane.ai/en/latest/introduction/configuration.html#format "
        "for futher detail."
    )

### Settings and Dataset

The number of dataset should be modified, otherwise too many data will queue for a long time. We choose the number of `num_bin_data = 100` such that the total number of data is

> `num_bin_data` $*$ `bin` $*$ `2 channels` $*$ $(1 - $`data_ratio`$)$
> 
> => $100 * 10 * 2 * (1-0.8)=400$

In [ ]:
shots = 1024
rnd_seed_range = range(3)

# Modify the configuration.
max_num_ptcs = 4
general_config["batch_size"] = 50
general_config["max_num_ptcs"] = max_num_ptcs
general_config["pad_num_ptcs"] = max_num_ptcs
general_config["num_ptcs_range"] = (max_num_ptcs, max_num_ptcs)
general_config["num_bin_data"] = 100

# Quantum configurations.
num_ir_qubits = int(np.ceil(np.log2(max_num_ptcs)))
num_nr_qubits = 7
num_layers = 1
num_reupload = 7

In [ ]:
data_config_list = [
    # 2-prong v.s. 1-prong.
    g_main.generate_data_config(
        sig="VzToZhToVevebb",
        bkg="VzToQCD",
        abbrev="BB-QCD",
        general_config=general_config,
    ),
    
    # 3-prong v.s. 1-prong.
    g_main.generate_data_config(
        sig="VzToTt",
        bkg="VzToQCD",
        abbrev="TT-QCD",
        general_config=general_config,
    ),
]

### Prediction function

In [ ]:
def prediction(x: torch.Tensor, data_config: dict, quantum_config: dict):
    """Predict with the pretrained model.

    Args:
        x : torch.Tensor
            Data to be predicted.
        data_config : dict
            Information about the data.
        quantum_config : dict
            Configurations of the quantum device.

    Returns:
        torch.Tensor -> Predictions.
    """

    # The `qidx` is fixed since we trained the model with `num_ir_qubits = 4`.
    model_suffix = f"qidx4_qnn{num_nr_qubits}_gl{num_layers}_gr{num_reupload}"

    # Get checkpoints file.
    data_suffix = f"{data_config['abbrev']}_ptc16_thres0.05-nb500_R0"
    ckpt_key = f"QCGNN_IX_{model_suffix}_SUM-{data_suffix}"
    ckpt_path = module_training.get_ckpt_path(ckpt_key, general_config["rnd_seed"])
    
    # Create model and load checkpoints.
    model = g_main.QuantumRotQCGNN(
        num_ir_qubits=num_ir_qubits,
        num_nr_qubits=num_nr_qubits,
        num_layers=num_layers,
        num_reupload=num_reupload,
        quantum_config=quantum_config,
        return_meas=True
    )

    # Load the pretrained model and set to evaluation mode.
    module_training.load_state_dict(model, ckpt_path)
    model.eval()
    
    return model(x)

### Run on IBMQ

A gmail will be sent from `maplexsendxemail@gmail.com` when the script is finished. See `except` and `else` sections in `try` for detail message of the email.

The email settings are saved in `./gmail.json` (which is default inavailable in `.gitignore`), the detail structure of the `./gmail.json` file is:

- "from": "maplexsendxemail@gmail.com",
- "to": "example@email.address",
- "passwd": "xxxxxxxxxxxxxxxx" (16-character password without spaces.)

In [ ]:
try:
    # Choose an IBMQ backend
    ibmq_backend = input("Enter IBMQ backend = ") or "ibmq_qasm_simulator"

    # Prediction on IBMQ.
    for data_config, rnd_seed in product(data_config_list, rnd_seed_range):
        
        # Seed everything.
        general_config["rnd_seed"] = rnd_seed
        L.seed_everything(general_config["rnd_seed"])
        
        # Create the data module.
        data_module = g_main.generate_datamodule(
            data_config=data_config,
            data_ratio=general_config["data_ratio"],
            batch_size=general_config["batch_size"],
            graph=False
        )

        # Buffers for saving data.
        x, y_true = torch.tensor([]), torch.tensor([])
        y_penl, meas_penl = torch.tensor([]), torch.tensor([])
        y_ibmq, meas_ibmq = torch.tensor([]), torch.tensor([])

        # Batch data and batch label will start with prefix "_".
        for _x, _y_true in tqdm(data_module.test_dataloader()):
            
            # Prediction on PennyLane.
            _y_penl, _meas_penl = prediction(
                x=_x,
                data_config=data_config,
                quantum_config={
                    "qdevice": "default.qubit",
                    "diff_method": "best",
                    "qbackend": "",
                }
            )

            # Prediction on IBMQ.
            _y_ibmq, _meas_ibmq = prediction(
                x=_x,
                data_config=data_config,
                quantum_config={
                    "qdevice": "qiskit.ibmq",
                    "diff_method": "parameter-shift",
                    "qbackend": ibmq_backend,
                }
            )

            # Dataset and true labels.
            x = torch.cat((x, _x))
            y_true = torch.cat((y_true, _y_true))
            
            # Save predictions to buffers.
            y_penl = torch.cat((y_penl, _y_penl))
            y_ibmq = torch.cat((y_ibmq, _y_ibmq))
            meas_penl = torch.cat((meas_penl, _meas_penl))
            meas_ibmq = torch.cat((meas_ibmq, _meas_ibmq))

        # Output dictionary    
        result = {
            "x": x, "y_true": y_true,
            "y_penl": y_penl, "meas_penl": meas_penl,
            "y_ibmq": y_ibmq, "meas_ibmq": meas_ibmq
        }

        # Save result to npy files.
        npy_file = (
            f"{ibmq_backend}-s{shots}-{data_config['abbrev']}_"
            f"r{general_config['rnd_seed']}-"
            f"p{general_config['max_num_ptcs']}_"
            f"n{general_config['num_bin_data']}_"
            f"b{general_config['batch_size']}-"
            f"{num_nr_qubits}_{num_layers}_{num_reupload}.npy"
        )
        npy_path = os.path.join(ibmq_dir, npy_file)
        np.save(npy_path, result, allow_pickle=True)

except Exception as e:
    subject = f"Failed to run ibmq on {ibmq_backend}"
    message = str(e)

else:
    model_suffix = f"qidx4_qnn{num_nr_qubits}_gl{num_layers}_gr{num_reupload}"
    subject = f"Finished running ibmq on {ibmq_backend}"
    message = f"Finish {model_suffix} with rnd_seed_range = {rnd_seed_range}"

finally:
    if os.path.isfile("gmail.json"):
        module_gmail.send_email(subject, message, config=general_config)
    else:
        print(subject)
        print(message)